#### **Lip Reading Application**

##### Install/Import dependencies

In [ ]:
%%capture
#Install dependencies and confirm installation with !pip list
!pip install opencv-python matplotlib imageio gdown tensorflow-macos
!pip list

In [ ]:
#Import dependencies
import os
import cv2
#import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
import tensorflow as tf